#### Test:

In [9]:
import dataset
from xai_cola import data,ml_model,counterfactual_explainer,cola,counterfactual_refiner
from xai_cola.main import COunterfactualwithLimitedActions

In [10]:
%reload_ext autoreload
%autoreload 2

In [11]:
dataset =dataset.GermanCreditDataset()
df = dataset.get_dataframe()
df.head(2)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,1,2,1,0,1,1169,6,5,0
1,22,0,2,1,1,2,5951,48,5,1


##### Configuration

In [12]:
data1 = data.PandasData(data=df, target_name="Risk")

In [13]:
ml_model1 = ml_model.Model(model_path='LGBMClassifier.pkl', backend='sklearn')  ## in this part, I want to change model_path to model in the furture.

In [14]:
counterfactual_explainer1 = counterfactual_explainer.DiCE(ml_model=ml_model1, data=data1, sample_num=4)  #will add more input params according to the dice_ml algorithm 

----LGBMClassifier.pkl model has been loaded----
---- predictions have been made----


100%|██████████| 4/4 [00:00<00:00,  7.34it/s]


----LGBMClassifier.pkl model has been loaded----
---- predictions have been made----


100%|██████████| 4/4 [00:00<00:00,  7.09it/s]


In [26]:
x_factual, x_counterfactual = counterfactual_explainer1.generate_counterfactuals()   #numpy类型

----LGBMClassifier.pkl model has been loaded----
---- predictions have been made----


100%|██████████| 4/4 [00:00<00:00,  4.03it/s]


array([[  46,    1,    2,    0,    1,    1, 3931,   48,    1],
       [  57,    1,    1,    1,    1,    1,  709,   12,    5],
       [  45,    1,    2,    0,    1,    1, 7882,   42,    4],
       [  32,    0,    1,    1,    2,    2,  931,    6,    1]],
      dtype=int64)

##### *choose policy and make limitation


In [20]:
ot_pshap_max = COunterfactualwithLimitedActions(data=data1, ml_model=ml_model1, explainer=counterfactual_explainer1, limited_actions=10, matcher="ot", attributor="pshap", Avalues_method="max")

In [22]:
factual, ce, ace = ot_pshap_max.generate_results()

----LGBMClassifier.pkl model has been loaded----
----LGBMClassifier.pkl model has been loaded----
----LGBMClassifier.pkl model has been loaded----
---- predictions have been made----
----LGBMClassifier.pkl model has been loaded----
---- predictions have been made----
----LGBMClassifier.pkl model has been loaded----
---- predictions have been made----


In [23]:
factual

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,30,1,3,1,1,2,4455,36,0,1
1,47,1,2,0,2,2,12612,36,3,1
2,22,0,1,1,1,1,806,15,0,1
3,23,1,1,1,3,2,2520,27,5,1


In [24]:
ce

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,30,1,0,1,1,2,8038,36,0,0
1,19,1,1,1,3,2,7651,27,5,0
2,22,1,1,1,1,1,806,15,4,0
3,65,1,2,0,2,2,1685,36,3,0


In [25]:
ace

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,19,1,3,1,3,2,7651,27,0,0
1,30,1,2,0,2,2,8038,36,3,0
2,22,1,1,1,1,1,806,15,0,0
3,65,1,1,1,3,2,1685,36,5,0
